In [1]:
import os
import xml.dom.minidom
import xml.etree.ElementTree as ET
from PIL import Image

### generate xmls from prediction, jpg is not padded to target size

In [ ]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().rsplit(' ', 5)  # split from right, in case there is blank space in jpg name
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xmin, ymin = int(float(tokens[2])), int(float(tokens[3]))
            xmax, ymax = int(float(tokens[4])), int(float(tokens[5]))
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml(basename, labels, data_path, save_path):
    jpg_name = os.path.join(data_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    need_write = False
    for label in labels:
        if label["det"] > 0.2:
            need_write = True
            object = ET.SubElement(root, "object")
            ET.SubElement(object, "name").text = label["name"]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")          
            ET.SubElement(bndbox, "xmin").text = str(label["xmin"])
            ET.SubElement(bndbox, "ymin").text = str(label["ymin"])
            ET.SubElement(bndbox, "xmax").text = str(label["xmax"])
            ET.SubElement(bndbox, "ymax").text = str(label["ymax"])

    if not need_write:
        return
    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))


def main(result_txt, class_i, data_path, save_path):
    os.makedirs(save_path, exist_ok=True)
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml(basename, labels, data_path, save_path)
        print("generated", basename+".xml")

In [ ]:
result_txt = "../../darknet/results/comp4_det_test_TRI.txt"
class_i = "TRI"
data_path = "../../yolo_tri_20181128/test"
save_path = "../../yolo_tri_20181128/test_pred"

main(result_txt, class_i, data_path, save_path)

### generate xmls from prediction, jpg is padded to target size

In [2]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().rsplit(' ', 5)  # split from right, in case there is blank space in jpg name
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xmin, ymin = int(float(tokens[2])), int(float(tokens[3]))
            xmax, ymax = int(float(tokens[4])), int(float(tokens[5]))
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml_with_padding(basename, labels, orig_path, pad_path, save_path):
    jpg_name = os.path.join(orig_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    pad_name = os.path.join(pad_path, basename+".txt")
    with open(pad_name, 'r') as f:
        line = f.readline()
        tokens = line.strip().split()
        padx, pady = int(tokens[1]), int(tokens[3])
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    need_write = False
    for label in labels:
        if label["det"] > 0.2:
            need_write = True
            object = ET.SubElement(root, "object")
            ET.SubElement(object, "name").text = label["name"]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")          
            ET.SubElement(bndbox, "xmin").text = str(label["xmin"]+padx)
            ET.SubElement(bndbox, "ymin").text = str(label["ymin"]+pady)
            ET.SubElement(bndbox, "xmax").text = str(label["xmax"]+padx)
            ET.SubElement(bndbox, "ymax").text = str(label["ymax"]+pady)

    if not need_write:
        return
    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))


def main(result_txt, class_i, orig_path, pad_path, save_path):
    os.makedirs(save_path, exist_ok=True)
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml_with_padding(basename, labels, orig_path, pad_path, save_path)
        print("generated", basename+".xml")

In [3]:
result_txt = "../../darknet/results/comp4_det_test_TRI.txt"
class_i = "TRI"
orig_path = "../../yolo_tri_20181128/test"
pad_path = "../../yolo_tri_20181128/test608"
save_path = "../../yolo_tri_20181128/test_pred"

main(result_txt, class_i, orig_path, pad_path, save_path)

generated 2018-06-01_12_40_20_x40468_y30353_w64_h64.xml
generated NJ18088265_x21990_y32694_w48_h48.xml
generated NJ18088247_x23335_y37018_w70_h58.xml
generated 2018-06-01_12_40_20_x45157_y34658_w64_h62.xml
generated NJ18088152_x32581_y5458_w68_h48.xml
generated 2018-06-01_12_40_20_x34904_y47955_w56_h56.xml
generated 2018-06-01_12_40_20_x24769_y47599_w70_h62.xml
generated NJ18088247_x18251_y16745_w62_h54.xml
generated NJ18088034_x32020_y14008_w60_h72.xml
generated 2018-06-01_12_40_20_x29805_y14444_w62_h60.xml
generated NJ18088301_x18847_y9182_w62_h50.xml
generated 2018-06-01_12_40_20_x19385_y34383_w52_h54.xml
generated 2018-06-01_12_40_20_x20956_y41231_w52_h62.xml
generated NJ18088302_x17456_y43571_w54_h60.xml
generated NJ18088753_x19489_y35790_w56_h66.xml
generated NJ18088753_x10520_y8804_w62_h56.xml
generated NJ18088034_x46416_y24908_w74_h60.xml
generated NJ18088152_x12807_y10391_w46_h48.xml
generated 2018-06-01_12_40_20_x6188_y22013_w76_h60.xml
generated NJ18088753_x7086_y32719_w66_h

generated NJ18088753_x11760_y41665_w74_h66.xml
generated 2018-06-01_12_40_20_x8774_y19968_w48_h84.xml
generated NJ18088753_x11754_y31096_w74_h56.xml
generated NJ18088265_x43117_y37736_w46_h52.xml
generated NJ18088152_x37098_y12391_w66_h62.xml
generated NJ18088301_x17374_y26614_w64_h54.xml
generated NJ18088094_x31437_y46690_w56_h38.xml
generated 2018-06-01_12_40_20_x26608_y17225_w56_h58.xml
generated NJ18088152_x42922_y33606_w64_h66.xml
generated 2018-06-01_12_40_20_x35708_y13242_w64_h74.xml
generated 2018-06-01_12_40_20_x22082_y32583_w68_h66.xml
generated NJ18088152_x45190_y26708_w54_h64.xml
generated 2018-06-01_12_40_20_x51294_y35901_w60_h58.xml
generated NJ18088247_x7685_y23981_w46_h46.xml
generated NJ18088265_x23095_y37119_w44_h46.xml
generated NJ18088265_x44026_y29592_w70_h52.xml
generated 2018-06-01_12_40_20_x34472_y32248_w74_h62.xml
generated NJ18088152_x26536_y16052_w70_h58.xml
generated 2018-06-01_12_40_20_x7815_y37115_w106_h76.xml
generated NJ18088265_x14425_y26516_w66_h54.xml

generated NJ18088265_x42358_y15369_w54_h58.xml
generated 2018-06-01_12_40_20_x9273_y28736_w52_h50.xml
generated NJ18088034_x31999_y15792_w64_h64.xml
generated 2018-06-01_12_40_20_x20663_y28566_w64_h70.xml
generated 2018-06-01_12_40_20_x31176_y37827_w70_h70.xml
generated 2018-06-01_12_40_20_x34613_y50107_w64_h56.xml
generated NJ18088265_x18052_y38755_w40_h50.xml
generated 2018-06-01_12_40_20_x26624_y18139_w74_h62.xml
generated NJ18088034_x40333_y14573_w66_h58.xml
generated 2018-06-01_12_40_20_x44209_y46941_w56_h56.xml
generated NJ18088152_x31600_y46377_w46_h52.xml
generated NJ18088753_x6848_y21002_w56_h70.xml
generated NJ18088152_x11544_y34578_w68_h62.xml
generated 2018-06-01_12_40_20_x32079_y36734_w62_h56.xml
generated NJ18088301_x11544_y32733_w66_h68.xml
generated 2018-06-01_12_40_20_x49485_y39673_w62_h62.xml
generated NJ18088265_x40947_y46402_w64_h40.xml
generated NJ18088265_x33626_y16846_w46_h46.xml
generated NJ18088302_x48822_y19221_w58_h64.xml
generated NJ18088034_x22008_y20263_w7

generated NJ18088152_x42732_y29775_w82_h56.xml
generated 2018-06-01_12_40_20_x9956_y28385_w86_h56.xml
generated 2018-06-01_12_40_20_x24437_y48887_w62_h62.xml
generated 2018-06-01_12_40_20_x6281_y33809_w44_h52.xml
generated 2018-06-01_12_40_20_x9134_y27413_w62_h44.xml
generated 2018-06-01_12_40_20_x8594_y38776_w98_h76.xml
generated NJ18088265_x18209_y12953_w56_h44.xml
generated 2018-06-01_12_40_20_x31436_y40727_w62_h62.xml
generated 2018-06-01_12_40_20_x45105_y43705_w54_h62.xml
generated 2018-06-01_12_40_20_x19269_y12784_w68_h64.xml
generated 2018-06-01_12_40_20_x46653_y40801_w64_h62.xml
generated NJ18088034_x34240_y16469_w62_h64.xml
generated NJ18088152_x40297_y10942_w44_h42.xml
generated 2018-06-01_12_40_20_x43020_y25095_w66_h66.xml
generated 2018-06-01_12_40_20_x9058_y16424_w82_h80.xml
generated NJ18088302_x19849_y6880_w66_h68.xml
generated 2018-06-01_12_40_20_x46860_y39929_w68_h68.xml
generated 2018-06-01_12_40_20_x8066_y36538_w78_h52.xml
generated NJ18088152_x32268_y32628_w48_h56.x

generated 2018-06-01_12_40_20_x32508_y14462_w50_h66.xml
generated 2018-06-01_12_40_20_x47201_y38765_w58_h72.xml
generated 2018-06-01_12_40_20_x6264_y19854_w54_h64.xml
generated 2018-06-01_12_40_20_x24189_y41550_w62_h62.xml
generated 2018-06-01_12_40_20_x32622_y30815_w58_h66.xml
generated 2018-06-01_12_40_20_x6586_y27127_w96_h74.xml
generated NJ18088152_x39709_y27699_w60_h64.xml
generated NJ18088265_x38306_y26555_w52_h44.xml
generated NJ18088247_x43120_y41226_w84_h70.xml
generated NJ18088152_x33461_y44793_w58_h68.xml
generated 2018-06-01_12_40_20_x40210_y30241_w66_h78.xml
generated 2018-06-01_12_40_20_x36033_y19863_w56_h52.xml
generated NJ18088302_x28579_y46048_w66_h64.xml
generated NJ18088302_x29809_y43804_w52_h54.xml
generated NJ18088302_x26163_y32415_w66_h52.xml
generated NJ18088265_x35392_y36770_w54_h42.xml
generated 2018-06-01_12_40_20_x32699_y9781_w66_h54.xml
generated 2018-06-01_12_40_20_x28449_y39868_w60_h60.xml
generated 2018-06-01_12_40_20_x32079_y28530_w62_h76.xml
generated N

...

generated NJ18088302_x50212_y29790_w78_h56.xml
generated NJ18088034_x18194_y6455_w78_h52.xml
generated NJ18088302_x51249_y35883_w66_h58.xml
generated NJ18088265_x44803_y35812_w68_h54.xml
generated NJ18088301_x25921_y15097_w44_h36.xml
generated 2018-06-01_12_40_20_x8456_y37863_w66_h78.xml
generated 2018-06-01_12_40_20_x8254_y30355_w62_h54.xml
generated 2018-06-01_12_40_20_x23085_y42102_w62_h54.xml
generated NJ18088265_x38729_y21276_w54_h58.xml
generated NJ18088039_x53088_y37560_w54_h66.xml
generated NJ18088034_x19041_y24556_w44_h46.xml
generated NJ18088301_x46426_y33317_w70_h64.xml
generated NJ18088301_x39507_y37390_w60_h64.xml
generated 2018-06-01_12_40_20_x34570_y22322_w68_h60.xml
generated 2018-06-01_12_40_20_x7374_y37353_w74_h68.xml
generated NJ18088094_x40145_y39239_w68_h64.xml
generated NJ18088753_x16230_y18505_w64_h56.xml
generated NJ18088265_x42110_y19026_w64_h60.xml
generated 2018-06-01_12_40_20_x9442_y18536_w96_h68.xml
generated 2018-06-01_12_40_20_x9560_y17529_w72_h58.xml
gen

generated 2018-06-01_12_40_20_x33467_y22339_w54_h56.xml
generated 2018-06-01_12_40_20_x8152_y27092_w56_h84.xml
generated NJ18088152_x23492_y15180_w68_h58.xml
generated 2018-06-01_12_40_20_x38316_y27754_w66_h60.xml
generated NJ18088301_x18453_y11042_w68_h58.xml
generated 2018-06-01_12_40_20_x41321_y19108_w58_h54.xml
generated 2018-01-15-15_13_04_x44198_y39380_w66_h74.xml
generated 2018-06-01_12_40_20_x8497_y19030_w68_h62.xml
generated NJ18088247_x14066_y19308_w62_h58.xml
generated NJ18088265_x8495_y26530_w46_h44.xml
generated NJ18088034_x41751_y44788_w68_h64.xml
generated NJ18088152_x32018_y17248_w40_h40.xml
generated 2018-06-01_12_40_20_x50495_y26415_w60_h78.xml
generated NJ18088152_x48450_y6689_w50_h36.xml
generated 2018-06-01_12_40_20_x21079_y37922_w62_h62.xml
generated 2018-06-01_12_40_20_x48668_y39752_w66_h62.xml
generated NJ18088265_x32294_y38481_w64_h64.xml
generated 2018-06-01_12_40_20_x38692_y19410_w64_h76.xml
generated NJ18088152_x44221_y33654_w48_h46.xml
generated 2018-06-01_

generated 2018-06-01_12_40_20_x9648_y27713_w60_h62.xml
generated 2018-06-01_12_40_20_x19222_y14703_w60_h62.xml
generated NJ18088152_x9518_y39919_w58_h56.xml
generated 2018-06-01_12_40_20_x7703_y22307_w62_h62.xml
generated 2018-06-01_12_40_20_x35088_y42354_w62_h48.xml
generated NJ18088265_x19906_y34053_w44_h42.xml
generated 2018-06-01_12_40_20_x18942_y29396_w60_h70.xml
generated NJ18088265_x21218_y15846_w74_h52.xml
generated 2018-06-01_12_40_20_x6922_y28147_w70_h72.xml
generated 2018-06-01_12_40_20_x9064_y18388_w76_h68.xml
generated NJ18088301_x19242_y5967_w64_h64.xml
generated 2018-06-01_12_40_20_x34992_y27892_w60_h56.xml
generated 2018-06-01_12_40_20_x52117_y33507_w60_h60.xml
generated 2018-06-01_12_40_20_x43051_y21417_w70_h64.xml
generated NJ18088152_x6102_y30385_w48_h54.xml
generated NJ18088034_x22050_y15777_w64_h54.xml
generated NJ18088753_x17623_y33119_w48_h46.xml
generated NJ18088302_x11108_y6270_w60_h64.xml
generated NJ18088302_x23535_y38304_w68_h66.xml
generated 2018-06-01_12_4

generated NJ18088247_x24091_y40807_w74_h78.xml
generated NJ18088094_x40133_y47129_w76_h58.xml
generated 2018-06-01_12_40_20_x9471_y33485_w94_h80.xml
generated NJ18088152_x30430_y26368_w60_h62.xml
generated NJ18088152_x22081_y15989_w42_h44.xml
generated NJ18088302_x40124_y43974_w60_h70.xml
generated 2018-06-01_12_40_20_x43023_y21175_w62_h80.xml
generated NJ18088301_x12548_y43841_w56_h60.xml
generated NJ18088301_x24292_y31508_w62_h50.xml
generated NJ18088034_x16202_y13160_w60_h72.xml
generated 2018-06-01_12_40_20_x50330_y25088_w68_h66.xml
generated NJ18088152_x45771_y42515_w64_h60.xml
generated 2018-06-01_12_40_20_x8921_y14091_w86_h70.xml
generated NJ18088034_x23915_y49465_w68_h60.xml
generated NJ18088302_x24130_y47268_w48_h48.xml
generated NJ18088152_x40660_y24420_w66_h62.xml
generated NJ18088301_x7675_y47781_w72_h60.xml
generated 2018-06-01_12_40_20_x8752_y22628_w72_h62.xml
generated 2018-06-01_12_40_20_x22296_y43807_w56_h66.xml
generated NJ18088152_x35920_y48432_w54_h60.xml
generated 

generated 2018-01-15-15_32_42_x28440_y12059_w90_h108.xml
generated 2018-06-01_12_40_20_x48280_y37868_w70_h58.xml
generated NJ18088034_x31443_y33049_w56_h58.xml
generated NJ18088302_x27176_y18668_w52_h56.xml
generated 2018-06-01_12_40_20_x31522_y18334_w70_h66.xml
generated 2018-06-01_12_40_20_x39176_y48231_w58_h68.xml
generated NJ18088152_x17427_y34216_w42_h46.xml
generated 2018-06-01_12_40_20_x9805_y38959_w82_h90.xml
generated NJ18088302_x48403_y47825_w64_h72.xml
generated NJ18088301_x46577_y49964_w64_h58.xml
generated NJ18088034_x27177_y16992_w64_h66.xml
generated 2018-06-01_12_40_20_x36756_y49075_w62_h58.xml
generated 2018-06-01_12_40_20_x29404_y16996_w66_h60.xml
generated NJ18088302_x43571_y55940_w66_h58.xml
generated 2018-06-01_12_40_20_x8541_y37738_w64_h118.xml
generated 2018-06-01_12_40_20_x9624_y14639_w60_h66.xml
generated 2018-06-01_12_40_20_x45776_y37528_w64_h66.xml
generated 2018-06-01_12_40_20_x38459_y46220_w66_h68.xml
generated 2018-01-15-15_13_04_x30771_y23313_w118_h112.xm